# RDA5820 Functional Test 
- https://datasheet.lcsc.com/szlcsc/RDA-Microelectronics-RDA5820NS_C167244.pdf
- http://news.eeworld.com.cn/mcu/article_2017011333300.html

In [1]:
import os, sys

def gen_relative_path(target_path): 
    
    def del_same_parents(target_path, current_path):
        if len(target_path) * len(current_path) > 0:
            if target_path[0] == current_path[0]:
                target_path.pop(0)
                current_path.pop(0)
                del_same_parents(target_path, current_path)
            
    current_path = os.getcwd().replace('\\', '/').split('/')
    target_path = target_path.replace('\\', '/').split('/')
    del_same_parents(target_path, current_path)
    
    return ['..'] * len(current_path) + target_path


def append_source_relative_path(source_relative_path):
    sys.path.append(os.sep.join(source_relative_path))

In [2]:
# paths = [' ']
# paths = [gen_relative_path(p) for p in paths]
# print(paths)

In [3]:
paths =[['..', '..', '..', '..', '..', 'Signal Generators', 'bitbucket', 'github', 'codes'],
        ['..', '..', '..', '..', '..', 'FX2LP', 'bitbucket', 'github', 'codes'], 
        ['..', '..', '..', '..', '..', 'Utilities', 'bitbucket', 'github', 'codes'],
        ['..', '..', 'codes']]

for path in paths:
    append_source_relative_path(path)

In [4]:
%pylab inline
 
from array import array
from utilities.adapters import peripherals
from fm_transceivers import RDA5820
import fx2lp

Populating the interactive namespace from numpy and matplotlib


## Debug mode?

In [5]:
cls = RDA5820

cls.DEBUG_MODE_SHOW_BUS_DATA = False         # whether to show bus data. 
cls.DEBUG_MODE_PRINT_REGISTER = False        # whether to print registers. 

## Generators

In [6]:
with_hardware_device = False

if with_hardware_device:
    _i2c = peripherals.I2C.get_Ftdi_i2c()
else:
    _i2c = None  # using None for testing without actual hardware device.

    
bus = peripherals.I2C(_i2c)


****** Virtual device. Data may not be real ! ******



In [8]:
bus = fx2lp.I2C(as_400KHz = True)

In [9]:
# freq = 88.80e6
# freq = 97.7e6

rda = RDA5820(bus, freq = 97.7e6, work_mode = 'FM_Receiver', tx_power_dBm = 3)
# rda = RDA5820(bus, freq = 88.80e6, work_mode = 'FM_Transmitter', tx_power_dBm = 3)
initial_registers_values = rda.registers_values

## Compare configs

In [ ]:
rda.set_work_mode('Audio_Amplifier')

In [10]:
rda.set_work_mode(mode = 'FM_Transmitter')
rda.tuner.set_frequency(88.8e6)

In [ ]:
rda.set_work_mode('FM_Receiver')
rda.tuner.set_frequency(97.7e6)

## Noise Level Measurement

In [ ]:
plt.figure(figsize = (15, 5))
 
# noise_levels = rda.receiver.scan_noise_levels(freq_start = 88.9e6, freq_end = 90e6, wait_seconds = 0.2)
noise_levels = rda.receiver.scan_noise_levels()
nl = np.array(noise_levels).T
plt.plot(nl[0] / 1e6, nl[1]);
# plt.bar(nl[0] / 1e6, nl[1], orientation = 'vertical');

In [ ]:
noise_levels_sorted = sorted(noise_levels, key = lambda x: x[1])
noise_levels_sorted

## RDS

In [ ]:
rda.rds.set_rds(program_id = 0x0520,
                station_name = "Wei Lin",
                radio_text = "My Radio Station !",
                program_type_code = 4,
                repeat_count = 3, message_count = 1, rds_mix_ratio = 50,  
                rds_fifo_size = 20,
                enable = True)

In [ ]:
# while True:
#     si.set_rds_buffer(message = time.strftime("%Y-%m-%d %H:%M:%S"), use_FIFO = True)
#     time.sleep(10)

In [ ]:
si.set_rds(enable = False)

## Member functions test

In [ ]:
# for f in dir(RDA5820):
#     if not f.startswith('_'):
#         if f.islower():
#             if callable(getattr(Si4713, f)):
#                 print('rda.{}()'.format(f))

In [ ]:
DEFAULT_REGISTERS_VALUES = ( )

In [ ]:
si.init()
initial_registers_values = si.registers_values

In [ ]:
si.read_all_registers()
read_back_registers_values = si.registers_values

In [ ]:
df = si.map.compare_values_sets_pd(initial_registers_values, DEFAULT_REGISTERS_VALUES)
df[df.different == 1]

In [ ]:
df = si.map.compare_values_sets_pd(initial_registers_values, read_back_registers_values)
# df
df[df.different == 1]